In [87]:
import numpy as np
import random as rnd
from tqdm import tqdm

rnd.seed(42)

In [88]:
class Maze:

    matrix = np.array
    connectivity = int
    width = int
    height = int

    def __init__(self, path):
        import numpy as np
        import random as rnd
        rnd.seed(42)

        with open(path, 'r') as maze:
            maze = [int(x) for x in maze.readline().split()]
        width, height = maze[0:2]
        connectivity = maze[2]
        maze = maze[3:]
        matrix = []

        for y in range(height):
            matrix.append(maze[0 + height*y : width + height*y])

        self.matrix = np.array(matrix)
        self.width, self.height, self.connectivity, self.shape = \
            width, height, connectivity, (width, height) #NOTE: np.array.shape is (height, width), don't confuse those, although doesn't matter for square labyrinths

        self.target_loc = np.unravel_index(np.argmax(self.matrix, axis=None), self.matrix.shape)


    def up_frame(self, cell_code: int):
        return cell_code//1%2==1

    def right_frame(self, cell_code: int):
        return cell_code//2%2==1

    def bottom_frame(self, cell_code: int):
        return cell_code//4%2==1

    def left_frame(self, cell_code: int):
        return cell_code//8%2==1


    def up_frame_coords(self, coordinates: tuple[int, int]):
        x, y = coordinates
        cell_code = self.matrix[x][y]
        return cell_code//1%2==1

    def right_frame_coords(self, coordinates: tuple[int, int]):
        x, y = coordinates
        cell_code = self.matrix[x][y]
        return cell_code//2%2==1

    def bottom_frame_coords(self, coordinates: tuple[int, int]):
        x, y = coordinates
        cell_code = self.matrix[x][y]
        return cell_code//4%2==1

    def left_frame_coords(self, coordinates: tuple[int, int]):
        x, y = coordinates
        cell_code = self.matrix[x][y]
        return cell_code//8%2==1

In [89]:
class Ant:

    def __init__(self, x:int, y:int):
        self.position_init = (x, y)
        self.position = (x, y)
        self.route = [self.position_init]

    def reset(self):
        self.position = self.position_init
        self.route = [self.position_init]

In [90]:
class ACO:

    def __init__(self, maze:Maze, ant_num:int = None, evaporation_rate:float = 0.1,
                 criterion:str = "max_iter", route_param_limit:int = 15, max_iter:int = 1000,
                 exploitation_const:float = 1.0, trace_log_base:float = 1):
        self.maze = maze
        self.evaporation_rate = evaporation_rate
        self.criterion = criterion
        self.route_param_limit = route_param_limit
        self.max_iter = max_iter
        self.exploitation_const = exploitation_const
        self.trace_log_base = trace_log_base

        if ant_num is None:
            ant_num = maze.width*maze.height
        self.ant_num = ant_num
        ant_positions = rnd.sample([(x, y) for x in range(maze.width) for y in range(maze.height)], k=self.ant_num)
        self.ants = [Ant(pos[0], pos[1]) for pos in ant_positions]

        self.temp_trace_map = np.array([[0.0]*maze.width]*maze.height)
        self.trace_map = np.array([[1.0]*maze.width]*maze.height)

        self.depth = 0
        self.came2target = 0

    def clear_traces(self):
        self.temp_trace_map = np.array([[0.0]*self.maze.width]*self.maze.height)

    def move(self, ant:Ant, exploiting:bool = False):
        self.exploitation_const = 1
        x, y = ant.position
        if ant.position == self.maze.target_loc:
            return ant.position

        ways = [(x+1, y),
                (x-1, y),
                (x, y+1),
                (x, y-1)]
        checks = [self.maze.bottom_frame_coords,
                  self.maze.up_frame_coords,
                  self.maze.right_frame_coords,
                  self.maze.left_frame_coords]

        ways = [ways[i] for i in range(len(ways)) if not (checks[i](ant.position) or ways[i] in ant.route)]

        if len(ways) == 0:
            return None

        if self.maze.target_loc in ways:
            final_way = self.maze.target_loc
            self.came2target += 1
        else:
            weights = [self.trace_map[way]**self.exploitation_const for way in ways]
            final_way = rnd.choices(population=ways, weights=weights, k=1)[0]
            if exploiting:
                final_way = ways[np.argmax(weights)]

        ant.position = final_way
        ant.route.append(final_way)
        return ant.position

    def fit(self):
        for _ in tqdm(range(self.max_iter)):
            if sum([int(aco.get_route((x, y))[2])
                    for x in range(aco.maze.width)
                    for y in range(aco.maze.height)])\
                    /(self.maze.height * aco.maze.width) == 1 and self.criterion == "connectivity":
                break
            if self.criterion == "avg_route" and sum([int(aco.get_route((x, y))[1])
                                                      for x in range(aco.maze.width)
                                                      for y in range(aco.maze.height)])\
                    /(self.maze.height * aco.maze.width) == 1:
                break
            if self.criterion == "max_route" and self.depth < self.route_param_limit:
                break
            if min([int(aco.get_route((x, y))[1])
                    for x in range(aco.maze.width)
                    for y in range(aco.maze.height)]) == 1 < self.route_param_limit and self.criterion == "min_route":
                break

            for ant in self.ants:
                self.trace_log_base = 0.1
                while self.move(ant) not in [None, self.maze.target_loc]:
                    continue
                route = ant.route
                n = len(route)

                if self.depth<n:
                    self.depth = n

                if self.maze.target_loc in route:
                    self.trace_log_base = 1000.0

                for point in route:
                    self.temp_trace_map[point] += (self.trace_log_base**2)/n
                ant.reset()

            self.trace_map = self.trace_map + self.temp_trace_map
            self.clear_traces()
            self.trace_map = self.trace_map * (1-self.evaporation_rate)

        self.trace_map = (self.trace_map - np.min(self.trace_map))/np.max(self.trace_map)
        return self.trace_map

    def get_route(self, coordinates: tuple[int, int]):
        x, y = coordinates
        explorer_ant = Ant(x, y)

        while self.move(explorer_ant, exploiting=True) not in [None, self.maze.target_loc]:
            continue
        return explorer_ant.route, len(explorer_ant.route), explorer_ant.position==self.maze.target_loc, self.depth

In [91]:
M = Maze("mazes/M25_1.mz")
aco = ACO(M, criterion="connectivity")

aco.fit()

  1%|          | 7/1000 [00:01<03:03,  5.41it/s]


array([[3.03339425e-10, 3.63223326e-03, 4.75855341e-03, 2.46085132e-03,
        1.94740752e-03, 1.12173158e-02, 1.15697010e-02, 5.29734761e-04,
        2.46344926e-03, 2.82837701e-03, 3.19330456e-03, 4.18079898e-03,
        4.18079893e-03, 8.63175360e-03, 4.46188280e-03],
       [1.50731053e-03, 6.99097896e-03, 1.72431852e-03, 4.33046514e-03,
        7.70336684e-03, 1.05346975e-02, 9.83629428e-03, 1.29238039e-02,
        0.00000000e+00, 2.42521214e-03, 3.19330447e-03, 3.75646445e-03,
        6.24796169e-03, 1.44139325e-02, 4.14312270e-03],
       [7.51882031e-03, 6.99097847e-03, 5.54234177e-04, 1.62692008e-03,
        1.44273022e-02, 2.06224711e-02, 2.21566910e-02, 2.55447603e-02,
        2.02582474e-02, 1.37930121e-02, 8.67488944e-03, 6.91419382e-03,
        5.15462664e-03, 1.60576124e-02, 1.12266015e-03],
       [1.21346264e-02, 6.67496589e-03, 4.74264727e-03, 1.93741166e-03,
        1.30267378e-02, 1.83694875e-02, 3.23539286e-02, 1.71263487e-02,
        1.70138120e-02, 9.26234306e-0

In [92]:
sum([int(aco.get_route((x, y))[2]) for x in range(aco.maze.width) for y in range(aco.maze.height)])/225

1.0

In [93]:
aco.get_route((13, 14))

([(13, 14),
  (13, 13),
  (14, 13),
  (14, 12),
  (14, 11),
  (14, 10),
  (14, 9),
  (14, 8),
  (14, 7),
  (14, 6),
  (13, 6),
  (12, 6),
  (11, 6),
  (10, 6),
  (9, 6)],
 15,
 True,
 67)

In [282]:
M.matrix

array([[13,  1,  5,  3,  9,  5,  1,  7,  9,  5,  7, 13,  5,  3, 11],
       [ 9,  6,  9,  4,  0,  7, 12,  3,  8,  5,  7, 11, 11,  8,  6],
       [12,  3, 14,  9,  6,  9,  5,  4,  4,  7, 13,  4,  4,  0,  3],
       [ 9,  4,  7, 14, 13,  0,  5,  5,  1,  7,  9,  3, 13,  6, 10],
       [12,  5,  1,  7, 13,  4,  7,  9,  4,  3, 10, 12,  5,  5,  2],
       [ 9,  7, 14, 13,  5,  3,  9,  2, 13,  4,  6,  9,  5,  5,  6],
       [12,  5,  3, 29,  3, 10, 14, 14, 11,  9,  5,  6, 13,  1,  3],
       [ 9,  5,  0,  5,  0,  4,  1,  5,  4,  6, 11,  9,  5,  6, 10],
       [14, 13,  6, 11,  8,  7, 10, 11, 13,  1,  4,  4,  7,  9,  2],
       [13,  5,  1,  2, 12,  7,  8,  2, 13,  2,  9,  3, 11, 14, 10],
       [11,  9,  6,  8,  7,  9,  6, 12,  7, 12,  2, 14, 12,  5,  6],
       [ 8,  4,  3, 14, 13,  2, 13,  3, 13,  1,  6, 13,  1,  5,  3],
       [10, 11, 12,  5,  5,  0,  1,  4,  3,  8,  3, 13,  4,  3, 10],
       [ 8,  4,  5,  3,  9,  2, 12,  3, 14, 14, 12,  5,  5,  6, 10],
       [12,  7, 13,  6, 14, 12,  7

In [35]:
M.matrix[(0, 1)]

1

In [176]:
a = list(range(19))


ValueError: (1, 2) is not in list

In [230]:
np.arange(9.0).reshape((3, 3))

array([[0., 1., 2.],
       [3., 4., 5.],
       [6., 7., 8.]])

In [229]:
np.arange(3.0)

array([0., 1., 2.])

In [233]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.arange(9.0).reshape((3, 3))/2
x1+x2

array([[ 0. ,  1.5,  3. ],
       [ 4.5,  6. ,  7.5],
       [ 9. , 10.5, 12. ]])

In [34]:
from PIL import Image

img = Image.new( 'RGB', (255, 255), "white") # create a new black image
pixels = img.load() # create the pixel map

# for y in range(img.size[0]):    # for every pixel:
#     for x in range(10):
#         pixels[y, x] = (0,0,0)
    # for y in range(img.size[1]):
    #     pixels[i,j] = (i, j, 100) # set the colour accordingly

img.show()

In [35]:
def draw_up(img: Image):
    pixels = img.load()
    for y in range(img.size[0]):
        for x in range(10):
            pixels[y, x] = (0,0,0)
    return img

def draw_bottom(img: Image):
    pixels = img.load()
    for y in range(img.size[0]):
        for x in range(img.size[0]-10, img.size[0]):
            pixels[y, x] = (0,0,0)
    return img

def draw_right(img: Image):
    pixels = img.load()
    for y in range(10):
        for x in range(img.size[0]):
            pixels[y, x] = (0,0,0)
    return img

def draw_left(img: Image):
    pixels = img.load()
    for y in range(img.size[0]-10, img.size[0]):
        for x in range(img.size[0]):
            pixels[y, x] = (0,0,0)
    return img

In [36]:
a = draw_up(img)
b = draw_bottom(img)
c = draw_left(img)
d = draw_right(img)

In [38]:
d.show()